In [1]:
import tink
from tink import cleartext_keyset_handle
from tink import hybrid

In [2]:
# Register the hybrid encryption key managers. This is needed to create
# HybridEncrypt and HybridDecrypt primitives later.
hybrid.register()

In [3]:
# A private keyset created with
# tinkey create-keyset \
#   --key-template=DHKEM_X25519_HKDF_SHA256_HKDF_SHA256_AES_256_GCM \
#   --out private_keyset.cfg
# Note that this keyset has the secret key information in cleartext.
private_keyset = r"""{
    "key": [{
        "keyData": {
            "keyMaterialType":
                "ASYMMETRIC_PRIVATE",
            "typeUrl":
                "type.googleapis.com/google.crypto.tink.HpkePrivateKey",
            "value":
                "EioSBggBEAEYAhogVWQpmQoz74jcAp5WOD36KiBQ71MVCpn2iWfOzWLtKV4aINfn8qlMbyijNJcCzrafjsgJ493ZZGN256KTfKw0WN+p"
        },
        "keyId": 958452012,
        "outputPrefixType": "TINK",
        "status": "ENABLED"
    }],
    "primaryKeyId": 958452012
}"""

In [4]:
# The corresponding public keyset created with
# "tinkey create-public-keyset --in private_keyset.cfg"
public_keyset = r"""{
    "key": [{
        "keyData": {
            "keyMaterialType":
                "ASYMMETRIC_PUBLIC",
            "typeUrl":
                "type.googleapis.com/google.crypto.tink.HpkePublicKey",
            "value":
                "EgYIARABGAIaIFVkKZkKM++I3AKeVjg9+iogUO9TFQqZ9olnzs1i7Sle"          },
        "keyId": 958452012,
        "outputPrefixType": "TINK",
        "status": "ENABLED"
    }],
    "primaryKeyId": 958452012
}"""

In [5]:
# Create a keyset handle from the keyset containing the public key. Because
# this keyset does not contain any secrets, we can use
# `tink.read_no_secret_keyset_handle`.
public_keyset_handle = tink.read_no_secret_keyset_handle(
    tink.JsonKeysetReader(public_keyset)
)

In [6]:
# Retrieve the HybridEncrypt primitive from the keyset handle.
enc_primitive = public_keyset_handle.primitive(hybrid.HybridEncrypt)

In [7]:
# Use enc_primitive to encrypt a message. In this case the primary key of the
# keyset will be used (which is also the only key in this example).
ciphertext = enc_primitive.encrypt(b'message', b'context_info')
ciphertext.hex()

'013920d12ce3ff78cf4c941cff35ccf3bda5723413b8cd50548472e287507e9d3f1852ff6f78ea0e52c58850d8f73029d6f3ab06ca4e417cb90ba949'

In [8]:
# Create a keyset handle from the private keyset. The keyset handle provides
# abstract access to the underlying keyset to limit the exposure of accessing
# the raw key material. WARNING: In practice, it is unlikely you will want to
# use a cleartext_keyset_handle, as it implies that your key material is
# passed in cleartext which is a security risk.
private_keyset_handle = cleartext_keyset_handle.read(
    tink.JsonKeysetReader(private_keyset)
)

In [9]:
# Retrieve the HybridDecrypt primitive from the private keyset handle.
dec_primitive = private_keyset_handle.primitive(hybrid.HybridDecrypt)

In [10]:
# Use dec_primitive to decrypt the message. Decrypt finds the correct key in
# the keyset and decrypts the ciphertext. If no key is found or decryption
# fails, it raises an error.
decrypted = dec_primitive.decrypt(ciphertext, b'context_info')
decrypted

b'message'